In [592]:
import pandas as pd
from scipy.stats.mstats import gmean
import numpy as np

In [593]:
data_main = pd.read_excel("query-impala-5517.xlsx")

In [594]:
names = pd.read_excel("names.xlsx", dtype='string')
names = names.set_index('coicop_id').T.to_dict("list")
names

{'0111101': ['Pirinç'],
 '0111201': ['Un Buğday Unu'],
 '0111208': ['Bebek Maması (Toz Karışım)'],
 '0111209': ['Bulgur'],
 '0111301': ['Ekmek'],
 '0111401': ['Bisküvi'],
 '0111402': ['Kraker'],
 '0111404': ['Gofret'],
 '0111405': ['Pasta'],
 '0111408': ['Kek'],
 '0111411': ['Baklava'],
 '0111415': ['Ekmek Hamuru'],
 '0111501': ['Makarna'],
 '0111502': ['Şehriye'],
 '0111602': ['Tahıl Gevreği'],
 '0112201': ['Dana Eti'],
 '0112401': ['Koyun Eti'],
 '0112501': ['Kümes Hayvanları Tavuk'],
 '0112602': ['Sakatat'],
 '0112701': ['Sucuk'],
 '0112702': ['Sosis'],
 '0112703': ['Salam'],
 '0112709': ['Hazır et yemekleri'],
 '0113101': ['Balık'],
 '0113301': ['Konserve balik teneke kutuda ton baliği'],
 '0114101': ['Süt'],
 '0114301': ['Yoğurt'],
 '0114304': ['Hazır sütlü talılar'],
 '0114401': ['Peynir Beyaz Peynir'],
 '0114402': ['Peynir Kaşar Peyniri'],
 '0114403': ['Tulum peyniri'],
 '0114404': ['Krem peynir'],
 '0114501': ['Yumurta'],
 '0115101': ['Tereyağı'],
 '0115201': ['Margarin'],
 '01

In [595]:
# coicop id_si 'except' veya 'd' olan urunlerin coicop id'lerine ayırt 
# edilebilmeleri icin singular_product_id degerleri atanıyor

data_convert = data_main.copy()
data_convert['coicop_id'] = np.where(data_convert.coicop_id == 'except',data_convert.singular_product_id, data_convert.coicop_id)
data_convert['coicop_id'] = np.where(data_convert.coicop_id == 'd',data_convert.singular_product_id, data_convert.coicop_id)
data_convert.head()

,coicop_id,row_product_id,singular_product_id,product_name,product_code,stock_status,price,date_crawled
0,1161070100,05107966-5ad7-4653-b6ab-c70c7281a1b4,62800511,1 Kg Kabuklu Ceviz Yeni Mahsül,60186725,True,559.00,2021-01-03
1,1161070100,05107966-5ad7-4653-b6ab-c70c7281a1b4,62800511,1 Kg Kabuklu Ceviz Yeni Mahsül,60186725,True,559.00,2021-01-02
2,3212020200,168ea762-e9fd-4189-a4ab-9546c37de1ed,100335099,103189.Z Siyah Kadın Bot,NaN,True,199.99,2021-01-02
3,9540040302,87e83f5a-c016-458a-99b7-82cff18ba241,77106825,12 Li Akrilik Boya Beyazlı Set,91081461,True,69.00,2021-01-03
4,9540040302,87e83f5a-c016-458a-99b7-82cff18ba241,77106825,12 Li Akrilik Boya Beyazlı Set,91081461,True,69.00,2021-01-04


In [596]:
# bazi urunlerin coicop_id'si 'd' veya 'except' gibi string degerleri almis.
# bundan dolayi bu fonksiyon butun coicop_id'leri string veri tipinde donuyor

def convert_coicop(coicop_id):  # 11'li coicop_id'yi 7'li coicop_id'ye donusturur
    coicop_id = str(coicop_id)
    if len(coicop_id) < 11: # coicop_id uzunlugu bazi urunlerde 10 veya 6. Bunların basina 0 ekleniyor
        if coicop_id[0] not in ['e', 'd']:
            coicop_id = '0' + coicop_id
    return coicop_id[0: 7]

def get_name(coicop_id):  # coicop id'nin kategorisini doner
    try:
        return names[coicop_id][0]
    except:
        return "invalid coicop"

In [597]:
("11'li coicop id sayisi: ", len(data_convert.coicop_id.unique()))

("11'li coicop id sayisi: ", 551)

In [599]:
# 7'li coicop id'ler ekleniyor
data = data_convert.copy()
data = data.rename(columns={"coicop_id": "coicop_id_11"})
data.insert(1, 'coicop_id', 'none')
data.coicop_id = data.coicop_id_11.apply(convert_coicop)
data.price = abs(data.price + 1e-10)  # fiyati -1 veya 0 olan urunler var, geometrik ortalamada sorun cikariyorlar
data.head()

,coicop_id_11,coicop_id,row_product_id,singular_product_id,product_name,product_code,stock_status,price,date_crawled
0,1161070100,0116107,05107966-5ad7-4653-b6ab-c70c7281a1b4,62800511,1 Kg Kabuklu Ceviz Yeni Mahsül,60186725,True,559.00,2021-01-03
1,1161070100,0116107,05107966-5ad7-4653-b6ab-c70c7281a1b4,62800511,1 Kg Kabuklu Ceviz Yeni Mahsül,60186725,True,559.00,2021-01-02
2,3212020200,0321202,168ea762-e9fd-4189-a4ab-9546c37de1ed,100335099,103189.Z Siyah Kadın Bot,NaN,True,199.99,2021-01-02
3,9540040302,0954004,87e83f5a-c016-458a-99b7-82cff18ba241,77106825,12 Li Akrilik Boya Beyazlı Set,91081461,True,69.00,2021-01-03
4,9540040302,0954004,87e83f5a-c016-458a-99b7-82cff18ba241,77106825,12 Li Akrilik Boya Beyazlı Set,91081461,True,69.00,2021-01-04


In [600]:
print("7'li coicop id sayisi: ", len(data.coicop_id.unique()))

7'li coicop id sayisi:  408


In [619]:
# 7'li coicop'lara gore gruplanıp fiyatların geometrik ortalamasi alınıyor
data_grouped = data.groupby(['coicop_id', 'date_crawled'], axis=0, as_index=False).price.apply(gmean)
data_grouped = data_grouped.rename(columns={'price': 'gmean_price'}).reset_index(drop=True)
data_grouped.insert(1, 'name', 'none')
data_grouped.name = data_grouped.coicop_id.apply(get_name)
data_grouped = data_grouped[['coicop_id', 'name', 'gmean_price', 'date_crawled']]
data_grouped = data_grouped.sort_values(by=['date_crawled'])
len(data_grouped[data_grouped.date_crawled == '2020-10-06'])

127

In [609]:
# name degeri 'invalid coicop' olan girdilerin coicop_id sutunlarındaki 
# deger dingular_product_id degerleri
data_grouped.to_excel("veri2.xlsx", index=False)
print("excel kaydedildi.")

excel kaydedildi.
